<a href="https://colab.research.google.com/github/whaldsz/deep-learning/blob/main/Disease_Indicators_Prediction_v3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Disease Prediction



## Setup and initialization

In [55]:
import numpy as np
import pandas as pd
import random
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow import math

from sklearn.compose import make_column_transformer
from sklearn.preprocessing import MinMaxScaler, OneHotEncoder
from keras.utils.vis_utils import plot_model
import os

!pip install -q git+https://github.com/tensorflow/docs 


import tensorflow_docs as tfdocs
import tensorflow_docs.modeling
import tensorflow_docs.plots

print("TensorFlow version::", tf.__version__)


from sklearn.model_selection import train_test_split

TensorFlow version:: 2.11.0


  error: subprocess-exited-with-error
  
  git clone --filter=blob:none --quiet https://github.com/tensorflow/docs 'C:\Users\whald\AppData\Local\Temp\pip-req-build-w271at_g' did not run successfully.
  exit code: 128
  
  [1 lines of output]
  fatal: unable to access 'https://github.com/tensorflow/docs/': Could not resolve host: github.com
  [end of output]
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
error: subprocess-exited-with-error

git clone --filter=blob:none --quiet https://github.com/tensorflow/docs 'C:\Users\whald\AppData\Local\Temp\pip-req-build-w271at_g' did not run successfully.
exit code: 128

See above for output.

note: This error originates from a subprocess, and is likely not a problem with pip.


In [56]:
from  IPython import display
from matplotlib import pyplot as plt

import numpy as np

import pathlib
import shutil
import tempfile

In [57]:
# currentdir
logdir = os.path.join(os.getcwd(), "tensorboard_logs")
shutil.rmtree(logdir, ignore_errors=True)

In [58]:
# set random seed

tf.random.set_seed(74)

## 1. Dataset Preparation

In [59]:
disease_training = pd.read_csv('https://raw.githubusercontent.com/whaldsz/deep-learning/main/Disease_Prediction/Training.csv')
disease_testing = pd.read_csv('https://raw.githubusercontent.com/whaldsz/deep-learning/main/Disease_Prediction/Testing.csv')
disease_training.head()

URLError: <urlopen error [Errno 11001] getaddrinfo failed>

In [ ]:
#from google.colab import drive
#drive.mount('/content/drive')

#3 Remove last column

In [ ]:
disease_training.isna().sum()

In [ ]:
disease_training.drop('Unnamed: 133', inplace=True, axis=1)

disease_training.isna().sum()

In [ ]:
#disease_training.head()

## Convert category to numeric values

In [ ]:
#get class labels

class_names = np.unique(disease_training.prognosis)
disease_training.prognosis = pd.Categorical(disease_training.prognosis)
disease_testing.prognosis = pd.Categorical(disease_testing.prognosis)
class_names.shape

In [ ]:
#disease_training.prognosis.cat.codes
#disease_training
#disease_testing.head()

## Separate Features and Label - Training

### Training Set

In [ ]:
X = disease_training.drop('prognosis', axis=1)
y = disease_training.prognosis.cat.codes
np.unique(y)

### Unseen Test Set

In [ ]:
X_unseen = disease_testing.drop('prognosis', axis=1)
y_unseen = disease_testing.prognosis.cat.codes
np.unique(X_unseen)

## Split into Training & Validation Test

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=74)
#print(X_train.shape)
#print(y_train.shape)
#print(X_test.shape)
#print(y_test.shape)


In [ ]:
X_test

In [ ]:
# Number of features
FEATURES = X_test.shape[1]
FEATURES

## 2. Model Training

### Training configuration

In [ ]:
N_VALIDATION = int(1e3)
N_TRAIN = int(1e4)
BUFFER_SIZE = int(1e4)
BATCH_SIZE = 500
STEPS_PER_EPOCH = N_TRAIN//BATCH_SIZE

### Create Model

### Find the ideal learning rate

In [ ]:
lr_schedule = tf.keras.optimizers.schedules.InverseTimeDecay(
  0.001,
  decay_steps=STEPS_PER_EPOCH*1000,
  decay_rate=1,
  staircase=False)

def get_optimizer():
  return tf.keras.optimizers.Adam(lr_schedule)

In [ ]:
step = np.linspace(0,100000)
lr = lr_schedule(step)
plt.figure(figsize = (6,4))
plt.plot(step/STEPS_PER_EPOCH, lr)
plt.ylim([0,max(plt.ylim())])
plt.xlabel('Epoch')
_ = plt.ylabel('Learning Rate')

In [ ]:
#lrs = 1e-4 * (10 ** (tf.range(40)/20))
#plt.figure(figsize=(6,4))
#plt.semilogx(lrs, history.history['loss'])
#plt.xlabel("Learning Rate")
#plt.ylabel("Loss")
#plt.title("Learning Rate vs Loss")

### Settings for automation

In [ ]:
def get_callbacks(name):
  return [
    tfdocs.modeling.EpochDots(),
    tf.keras.callbacks.EarlyStopping(monitor='accuracy', patience=200),
    tf.keras.callbacks.TensorBoard(os.path.join(logdir,name)),
  ]

In [ ]:
def compile_and_fit(model, name, loss=None, optimizer=None, metrics = None, max_epochs=10000):
  if optimizer is None:
    optimizer = get_optimizer()

  if loss is None:
    loss = tf.keras.losses.BinaryCrossentropy(from_logits=True)
  if metrics is None:
    metrics = ['accuracy']
  model.compile(
      optimizer=optimizer,
      loss=loss,
      metrics=metrics
  )

  model.summary()

  history = model.fit(
    X_train,
    y_train,
    steps_per_epoch = STEPS_PER_EPOCH,
    epochs=max_epochs,
    validation_split=0.1,
    #validation_data=[X_test, y_test],
    callbacks=get_callbacks(name),
    verbose=0)
  return history

### Models

In [ ]:
size_histories = {}

#### Model 1

Simple model with 3 layers

In [ ]:
model1 = tf.keras.Sequential([
    layers.Dense(4, activation='elu', input_shape=(FEATURES,)),
    layers.Dense(41, activation=tf.keras.activations.softmax)
])

In [ ]:
size_histories['model1'] = compile_and_fit(
    model1, 
    'models/model1', 
    loss=tf.keras.losses.SparseCategoricalCrossentropy()
)

#### Model 2

In [ ]:
model2 = tf.keras.Sequential([
     layers.Dense(4, activation=tf.keras.activations.relu, input_shape=(FEATURES,)),
    layers.Dense(4, activation=tf.keras.activations.relu),
    layers.Dense(41, activation=tf.keras.activations.softmax)
])


size_histories['model2'] = compile_and_fit(
    model2, 
    'models/model2', 
    loss=tf.keras.losses.SparseCategoricalCrossentropy()
)

#model.compile(loss=tf.keras.losses.SparseCategoricalCrossentropy(),
#                optimizer = tf.keras.optimizers.Adam(),
#                #metrics=['MultiClassConfusionMatrixPlot'])
#                metrics=["accuracy"])

#scheduler = tf.keras.callbacks.LearningRateScheduler(lambda epoch: 1e-4 * 10 **(epoch/20))

#history = model.fit(X_train, y_train, epochs=40, callbacks=[scheduler])

#### Model 3

In [ ]:
model3 = tf.keras.Sequential([
     layers.Dense(4, activation=tf.keras.activations.relu, input_shape=(FEATURES,)),
    layers.Dense(4, activation=tf.keras.activations.relu),
    layers.Dense(4, activation=tf.keras.activations.relu),
    layers.Dense(41, activation=tf.keras.activations.softmax)
])


size_histories['model3'] = compile_and_fit(
    model3, 
    'models/model3', 
    loss=tf.keras.losses.SparseCategoricalCrossentropy()
)

#### Model 4

In [ ]:
model4 = tf.keras.Sequential([
     layers.Dense(4, activation=tf.keras.activations.relu, input_shape=(FEATURES,)),
    layers.Dense(4, activation=tf.keras.activations.relu),
    layers.Dense(4, activation=tf.keras.activations.relu),
    layers.Dense(4, activation=tf.keras.activations.relu),
    layers.Dense(41, activation=tf.keras.activations.softmax)
])


size_histories['model4'] = compile_and_fit(
    model4, 
    'models/model4', 
    loss=tf.keras.losses.SparseCategoricalCrossentropy()
)

#### Model 5

In [ ]:
model5 = tf.keras.Sequential([
     layers.Dense(4, activation=tf.keras.activations.relu, input_shape=(FEATURES,)),
    layers.Dense(4, activation=tf.keras.activations.relu),
    layers.Dense(4, activation=tf.keras.activations.relu),
    layers.Dense(4, activation=tf.keras.activations.relu),
    layers.Dense(4, activation=tf.keras.activations.relu),
    layers.Dense(41, activation=tf.keras.activations.softmax)
])


size_histories['model5'] = compile_and_fit(
    model5, 
    'models/model5', 
    loss=tf.keras.losses.SparseCategoricalCrossentropy()
)

### 3. Evaluate Model

#### Evaluate with test data

In [ ]:
loss1, acc1 = model1.evaluate(X_test, y_test)
loss2, acc2 = model2.evaluate(X_test, y_test)
loss3, acc3 = model3.evaluate(X_test, y_test)
loss4, acc4 = model4.evaluate(X_test, y_test)
loss5, acc5 = model5.evaluate(X_test, y_test)

print("Evaluation with Test Data")
print(f"Model 1 - Loss: {loss1}\t\tAccuracy: {acc1}")
print(f"Model 2 - Loss: {loss2}\t\tAccuracy: {acc2}")
print(f"Model 3 - Loss: {loss3}\t\tAccuracy: {acc3}")
print(f"Model 4 - Loss: {loss4}\t\tAccuracy: {acc4}")
print(f"Model 5 - Loss: {loss5}\t\tAccuracy: {acc5}")

#### Evaluate with unseen data

In [ ]:
loss1, acc1 = model1.evaluate(X_unseen, y_unseen)
loss2, acc2 = model2.evaluate(X_unseen, y_unseen)
loss3, acc3 = model3.evaluate(X_unseen, y_unseen)
loss4, acc4 = model4.evaluate(X_unseen, y_unseen)
loss5, acc5 = model5.evaluate(X_unseen, y_unseen)

print("Evaluation with Unseen Data")
print(f"Model 1 - Loss: {loss1}\t\tAccuracy: {acc1}")
print(f"Model 2 - Loss: {loss2}\t\tAccuracy: {acc2}")
print(f"Model 3 - Loss: {loss3}\t\tAccuracy: {acc3}")
print(f"Model 4 - Loss: {loss4}\t\tAccuracy: {acc4}")
print(f"Model 5 - Loss: {loss5}\t\tAccuracy: {acc5}")


## 3. Testing

#### Test Set

In [ ]:
### Test set
predictions = model.predict(X_test)
predictions[0]

In [ ]:
result = class_names[tf.argmax(predictions, axis=1)]


#### Unseen Test Data

In [ ]:
prediction_unseen_data = model.predict(X_unseen)

In [ ]:
result = class_names[tf.argmax(prediction_unseen_data, axis=1)]
result

### Correlation Matrix

In [ ]:
#plt.figure(figsize=(25,12))

#sns.heatmap(disease_training.corr().abs(), annot=True)
